In [1]:
# mobil_all_features.py
# Game "Mobil-mobilan" — Versi lengkap dengan banyak fitur tambahan
# Pastikan pygame terinstall: pip install pygame

import pygame
import random
import os
import json
import math
from collections import deque

# ----------------- KONFIG -----------------
WIDTH, HEIGHT = 540, 760
FPS = 60

LANES = 3
LANE_WIDTH = WIDTH // LANES

# Warna dasar
ROAD_COLOR_DAY = (40, 40, 40)
ROAD_COLOR_NIGHT = (18, 18, 28)
LINE_COLOR = (230, 230, 230)
BG_COLOR_DAY = (34, 139, 34)   # pinggir jalan hijau
BG_COLOR_NIGHT = (6, 40, 10)

PLAYER_COLOR = (20, 160, 240)
ENEMY_COLOR = (200, 20, 20)
SHIELD_COLOR = (255, 215, 0)
POWERUP_COLOR = (100, 255, 100)
COIN_COLOR = (255, 200, 20)

FONT_NAME = "freesansbold.ttf"
HIGHSCORE_FILE = "highscore.txt"
SAVE_FILE = "savegame.json"
LEADERBOARD_FILE = "leaderboard.json"

# Gameplay tuning
BASE_ENEMY_SPEED = 4.0
ENEMY_ACCEL = 0.12
SPAWN_INTERVAL = 900  # ms
POWERUP_CHANCE = 0.12
COIN_CHANCE = 0.18
SHIELD_DURATION = 3500  # ms
SLOWMO_DURATION = 3000
DOUBLE_SCORE_DURATION = 5000
NITRO_DURATION = 800  # ms for immediate boost
NITRO_COOLDOWN = 4000  # ms
NITRO_SPEED_MULT = 1.8

# Boss config
BOSS_EVERY_LEVELS = 5

# Save defaults
DEFAULT_SAVE = {
    "coins": 0,
    "unlocked_skins": ["default"],
    "selected_skin": "default",
    "highscore": 0
}

# -------------- HELPERS -------------------
def lane_center(index):
    left = index * LANE_WIDTH
    center = left + LANE_WIDTH // 2
    return center

def load_text_file(path, default=0):
    if not os.path.exists(path):
        return default
    try:
        with open(path, "r") as f:
            return int(f.read().strip())
    except:
        return default

def save_text_file(path, value):
    try:
        with open(path, "w") as f:
            f.write(str(value))
    except:
        pass

def load_json(path, default):
    if not os.path.exists(path):
        return default
    try:
        with open(path, "r") as f:
            return json.load(f)
    except:
        return default

def save_json(path, data):
    try:
        with open(path, "w") as f:
            json.dump(data, f, indent=2)
    except:
        pass

# -------------- GameObject ----------------
class GameObject(pygame.sprite.Sprite):
    def __init__(self, x, y, w, h):
        super().__init__()
        # Support transparency
        self.image = pygame.Surface((w, h), pygame.SRCALPHA)
        self.rect = self.image.get_rect(center=(x, y))
    def update(self, dt):
        pass

# -------------- Player --------------------
class Player(GameObject):
    def __init__(self, lane_index, skin="default"):
        w, h = 48, 88
        x = lane_center(lane_index)
        y = HEIGHT - 140
        super().__init__(x, y, w, h)
        self.lane = lane_index
        self.lives = 3
        self.is_shielded = False
        self.shield_end_time = 0
        self.score = 0
        self.speed_y = 0

        # Nitro
        self.nitro_ready_time = 0
        self.nitro_end_time = 0
        self.nitro_meter = 100  # 0..100
        self.nitro_max = 100
        self.nitro_recharge_rate = 8  # per second when not used

        # Visual / skin
        self.skin = skin
        self.turn_offset = 0  # tilt when moving left/right
        self.turn_target = 0

        # small invul after hit
        self.hit_flash_until = 0

        self._draw_base()

    def _draw_base(self):
        # Draw a car-like rectangle with a windshield — fallback if no sprite
        w, h = self.image.get_size()
        self.image.fill((0,0,0,0))
        pygame.draw.rect(self.image, PLAYER_COLOR, (6, 12, w-12, h-24), border_radius=8)
        pygame.draw.rect(self.image, (230,230,240), (w//2-10, 18, 20, 12), border_radius=3)
        # wheels
        pygame.draw.rect(self.image, (20,20,20), (10, h-18, 12, 8), border_radius=3)
        pygame.draw.rect(self.image, (20,20,20), (w-22, h-18, 12, 8), border_radius=3)

    def move_left(self):
        if self.lane > 0:
            self.lane -= 1
            self.turn_target = -8
            self.rect.centerx = lane_center(self.lane)

    def move_right(self):
        if self.lane < LANES - 1:
            self.lane += 1
            self.turn_target = 8
            self.rect.centerx = lane_center(self.lane)

    def boost(self):
        # small upward nudge and recharge nitro a bit
        self.rect.y -= 10
        if self.rect.top < 80:
            self.rect.top = 80

    def use_nitro(self, now):
        if now >= self.nitro_ready_time and self.nitro_meter >= 25:
            # activate nitro for short burst
            self.nitro_end_time = now + NITRO_DURATION
            self.nitro_ready_time = now + NITRO_COOLDOWN
            self.nitro_meter = max(0, self.nitro_meter - 25)
            return True
        return False

    def update(self, dt, now):
        # turn offset smoothing back to 0
        self.turn_offset += (self.turn_target - self.turn_offset) * min(1, dt / 80.0)
        if abs(self.turn_target - self.turn_offset) < 0.5:
            self.turn_target = 0

        # shield expiry
        if self.is_shielded and now > self.shield_end_time:
            self.is_shielded = False

        # nitro recharge
        if now > self.nitro_end_time:
            self.nitro_meter = min(self.nitro_max, self.nitro_meter + self.nitro_recharge_rate * (dt/1000.0))

        # flashing after hit
        if now < self.hit_flash_until:
            alpha = 120
            self.image.set_alpha(180)
        else:
            self.image.set_alpha(255)

# -------------- Enemy --------------------
class Enemy(GameObject):
    def __init__(self, lane_index, speed, etype="normal"):
        w, h = 48, 88
        x = lane_center(lane_index)
        y = -h - random.randint(0, 100)
        super().__init__(x, y, w, h)
        self.lane = lane_index
        self.base_speed = speed
        self.speed = speed
        self.etype = etype
        self.spawn_time = pygame.time.get_ticks()
         # tambah HP sesuai tipe musuh
        if self.etype == "boss":
            self.hp = 10
        elif self.etype == "big":
            self.hp = 3
        else:
            self.hp = 1
        self._draw_base()

        # zigzag specifics
        self.zig_amp = random.randint(18, 36)
        self.zig_freq = random.uniform(0.004, 0.02)

        # big enemy HP
        self.hp = 1
        if etype == "big":
            self.hp = 3
            self.image = pygame.Surface((w+20, h+20), pygame.SRCALPHA)
            self.rect = self.image.get_rect(center=(x, y))
            self._draw_base()
        elif etype == "boss":
            # boss much bigger and healthful
            self.hp = 12
            self.image = pygame.Surface((w*2.2, h*1.6), pygame.SRCALPHA)
            self.rect = self.image.get_rect(center=(x, y))
            self._draw_boss()
            self.osc = 0

    def _draw_base(self):
        # simple enemy car drawing
        self.image.fill((0,0,0,0))
        if self.etype == "fast":
            color = (255, 80, 80)
        elif self.etype == "big":
            color = (180, 60, 80)
        else:
            color = ENEMY_COLOR
        w, h = self.image.get_size()
        pygame.draw.rect(self.image, color, (6, 12, w-12, h-24), border_radius=8)
        pygame.draw.rect(self.image, (30,30,30), (w//2-10, 18, 20, 12), border_radius=3)
        # small health bar for big
        if self.etype == "big":
            pygame.draw.rect(self.image, (0,0,0,120), (6, 6, w-12, 6))
            for i in range(self.hp):
                pygame.draw.rect(self.image, (200,40,40), (8 + i*(w-14)/self.hp, 6, (w-14)/self.hp - 4, 6))

    def _draw_boss(self):
        w, h = self.image.get_size()
        self.image.fill((0,0,0,0))
        pygame.draw.rect(self.image, (180, 0, 0), (6, 12, w-12, h-24), border_radius=10)
        pygame.draw.rect(self.image, (40,40,40), (w//2-24, 20, 48, 16), border_radius=6)
        # boss eyes
        pygame.draw.circle(self.image, (255,255,60), (w-50, 30), 6)
        pygame.draw.circle(self.image, (255,255,60), (50, 30), 6)

    def update(self, dt):
        t = pygame.time.get_ticks()
        # variable speed adjustments
        if self.etype == "zigzag":
            # move down with x oscillation
            self.rect.y += self.speed * (dt / 16.67)
            shift = math.sin((t - self.spawn_time) * self.zig_freq) * self.zig_amp
            self.rect.centerx = lane_center(self.lane) + shift
        elif self.etype == "fast":
            self.rect.y += self.speed * (dt / 16.67) * 1.6
        elif self.etype == "big":
            self.rect.y += self.speed * (dt / 16.67) * 0.9
        elif self.etype == "boss":
            # slow approach, with horizontal oscillation
            self.osc += dt/1000.0
            self.rect.centerx = lane_center(self.lane) + math.sin(self.osc*1.8)*80
            self.rect.y += self.speed * (dt / 16.67) * 0.5
        else:
            self.rect.y += self.speed * (dt / 16.67)

# -------------- PowerUp ------------------
class PowerUp(GameObject):
    def __init__(self, lane_index, ptype="shield"):
        w, h = 36, 36
        x = lane_center(lane_index)
        y = -h - random.randint(0, 80)
        super().__init__(x, y, w, h)
        self.lane = lane_index
        self.speed = BASE_ENEMY_SPEED * 0.9
        self.ptype = ptype
        self._draw()

    def _draw(self):
        self.image.fill((0,0,0,0))
        if self.ptype == "shield":
            pygame.draw.circle(self.image, SHIELD_COLOR, (18,18), 16)
            pygame.draw.circle(self.image, (255,255,255), (18,18), 6)
        elif self.ptype == "slow":
            pygame.draw.circle(self.image, (120, 180, 255), (18,18), 16)
            pygame.draw.polygon(self.image, (255,255,255),( (12,10),(24,10),(18,26) ))
        elif self.ptype == "double":
            pygame.draw.circle(self.image, (255,210,60), (18,18), 16)
            self._draw_text("x2")
        elif self.ptype == "life":
            pygame.draw.circle(self.image, (255,100,100), (18,18), 16)
            pygame.draw.polygon(self.image,(255,255,255),[(9,18),(18,28),(27,18),(18,8)])
        elif self.ptype == "coin":
            pygame.draw.circle(self.image, COIN_COLOR, (18,18), 14)
            pygame.draw.circle(self.image, (255,240,160), (18,18), 6)

    def _draw_text(self, t):
        font = pygame.font.Font(FONT_NAME, 16)
        txt = font.render(t, True, (0,0,0))
        r = txt.get_rect(center=(18,18))
        self.image.blit(txt, r)

    def update(self, dt):
        self.rect.y += self.speed * (dt / 16.67)

# -------------- Particles (for effects) --------------
class Particle:
    def __init__(self, pos, vel, lifetime, color, size):
        self.pos = list(pos)
        self.vel = list(vel)
        self.lifetime = lifetime
        self.color = color
        self.size = size
        self.birth = pygame.time.get_ticks()

    def update(self, dt):
        self.pos[0] += self.vel[0] * (dt/16.67)
        self.pos[1] += self.vel[1] * (dt/16.67)
        self.lifetime -= dt

    def alive(self):
        return self.lifetime > 0

    def draw(self, surf):
        alpha = max(0, min(255, int(255 * (self.lifetime / 1000.0))))
        s = pygame.Surface((self.size*2, self.size*2), pygame.SRCALPHA)
        pygame.draw.circle(s, self.color + (alpha,), (self.size, self.size), self.size)
        surf.blit(s, (int(self.pos[0]-self.size), int(self.pos[1]-self.size)))

# -------------- Game Class -----------------
class Game:
    def __init__(self):
        pygame.init()
        pygame.mixer.init()
        pygame.display.set_caption("Mobil-mobilan - All Features")
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(FONT_NAME, 18)
        self.bigfont = pygame.font.Font(FONT_NAME, 36)
        self.smallfont = pygame.font.Font(FONT_NAME, 14)

        self.load_assets()
        self.reset()

    def load_assets(self):
        # try load sounds/images; if missing, continue gracefully
        self.sounds = {}
        try:
            self.sounds['hit'] = pygame.mixer.Sound("hit.wav")
            self.sounds['coin'] = pygame.mixer.Sound("coin.wav")
            self.sounds['powerup'] = pygame.mixer.Sound("powerup.wav")
            self.sounds['nitro'] = pygame.mixer.Sound("nitro.wav")
            self.sounds['bg'] = pygame.mixer.Sound("bg_loop.wav")
        except Exception:
            # no problem if absent
            pass

        # simple background music loop if available
        if 'bg' in self.sounds:
            try:
                self.sounds['bg'].play(-1)
            except:
                pass

    def reset(self):
        self.now = pygame.time.get_ticks()
        self.running = True
        self.playing = False
        self.paused = False
        self.in_shop = False

        # Groups
        self.all_sprites = pygame.sprite.Group()
        self.enemies = pygame.sprite.Group()
        self.powerups = pygame.sprite.Group()

        # Player
        start_lane = LANES // 2
        save = load_json(SAVE_FILE, DEFAULT_SAVE)
        skin = save.get("selected_skin", "default")
        self.player = Player(start_lane, skin=skin)
        self.all_sprites.add(self.player)

        # spawn
        self.last_spawn = self.now
        self.spawn_interval = SPAWN_INTERVAL
        self.enemy_speed = BASE_ENEMY_SPEED

        # state
        self.level = 1
        self.score = 0
        self.coins = save.get("coins", 0)
        self.highscore = save.get("highscore", load_text_file(HIGHSCORE_FILE))
        self.skill_multiplier = 1

        # visual
        self.road_offset = 0
        self.road_speed = 4

        # day-night
        self.time_of_day = 0.0  # 0..24 hours simil
        self.day_night_speed = 0.01

        # weather
        self.weather = None  # 'rain' or 'snow' or None
        self.weather_particles = []

        # particles for effects
        self.particles = []

        # difficulty curve
        self.spawn_deque = deque(maxlen=6)

        # powerup timers
        self.slowmo_until = 0
        self.double_until = 0

        # leaderboard
        self.leaderboard = load_json(LEADERBOARD_FILE, [])

        # ui touch area (if small screen)
        self.virtual = {
            "left": pygame.Rect(8, HEIGHT-140, 110, 130),
            "right": pygame.Rect(WIDTH-118, HEIGHT-140, 110, 130),
            "nitro": pygame.Rect(WIDTH//2-45, HEIGHT-120, 90, 90),
        }

    # ------------------ Spawn logic -------------------
    def spawn_enemy_or_powerup(self):
        lane = random.randrange(0, LANES)
        r = random.random()
        # powerups & coins
        if r < POWERUP_CHANCE:
            ptype = random.choice(["shield", "slow", "double", "life"])
            pu = PowerUp(lane, ptype=ptype)
            self.all_sprites.add(pu)
            self.powerups.add(pu)
            return
        if r < POWERUP_CHANCE + COIN_CHANCE:
            pu = PowerUp(lane, ptype="coin")
            self.all_sprites.add(pu)
            self.powerups.add(pu)
            return

        # enemy types by probability depending on level
        etype = "normal"
        roll = random.random()
        # higher level increases chance of fast/big/zigzag
        fast_ch = min(0.18 + self.level*0.01, 0.35)
        big_ch = min(0.12 + self.level*0.005, 0.22)
        zig_ch = min(0.14 + self.level*0.008, 0.28)

        if roll < fast_ch:
            etype = "fast"
        elif roll < fast_ch + big_ch:
            etype = "big"
        elif roll < fast_ch + big_ch + zig_ch:
            etype = "zigzag"
        else:
            etype = "normal"

        # boss spawn occasionally
        if self.level % BOSS_EVERY_LEVELS == 0 and random.random() < 0.12:
            etype = "boss"

        speed = self.enemy_speed + random.uniform(-0.6, 0.6)
        enemy = Enemy(lane, speed, etype=etype)
        self.all_sprites.add(enemy)
        self.enemies.add(enemy)

    # ------------------ Game loop -------------------
    def start(self):
        self.playing = True
        self.run()

    def run(self):
        while self.running:
            dt = self.clock.tick(FPS)
            self.now = pygame.time.get_ticks()
            self.handle_events()
            if self.playing and not self.paused and not self.in_shop:
                self.update(dt)
            self.draw()

    # ------------------ Events -----------------------
    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.quit()

            if event.type == pygame.KEYDOWN:
                if not self.playing:
                    if event.key == pygame.K_RETURN:
                        self.reset()
                        self.start()
                    elif event.key == pygame.K_s:
                        # open shop from main menu
                        self.in_shop = True
                    elif event.key == pygame.K_ESCAPE:
                        self.quit()
                else:
                    if event.key == pygame.K_LEFT:
                        self.player.move_left()
                    elif event.key == pygame.K_RIGHT:
                        self.player.move_right()
                    elif event.key == pygame.K_UP:
                        self.player.boost()
                    elif event.key == pygame.K_p:
                        self.paused = not self.paused
                    elif event.key == pygame.K_SPACE:
                        # cheat shield
                        self.activate_shield(SHIELD_DURATION)
                    elif event.key == pygame.K_LSHIFT or event.key == pygame.K_RSHIFT:
                        # nitro attempt
                        used = self.player.use_nitro(self.now)
                        if used:
                            try:
                                if 'nitro' in self.sounds: self.sounds['nitro'].play()
                            except: pass

            if event.type == pygame.MOUSEBUTTONDOWN:
                mx,my = event.pos
                # Virtual control for touchscreen
                if self.playing:
                    if self.virtual["left"].collidepoint(mx,my):
                        self.player.move_left()
                    elif self.virtual["right"].collidepoint(mx,my):
                        self.player.move_right()
                    elif self.virtual["nitro"].collidepoint(mx,my):
                        used = self.player.use_nitro(self.now)
                        if used:
                            try:
                                if 'nitro' in self.sounds: self.sounds['nitro'].play()
                            except: pass
                else:
                    # main menu buttons via click
                    # click center to start
                    if 0 <= mx <= WIDTH and 0 <= my <= HEIGHT:
                        self.reset()
                        self.start()

    # ------------------ Update -----------------------
    def update(self, dt):
        now = self.now
        # spawn logic, adjusted by level and possible slowmo
        effective_spawn_interval = self.spawn_interval
        if now - self.last_spawn > effective_spawn_interval:
            self.spawn_enemy_or_powerup()
            self.last_spawn = now
            jitter = random.randint(-120, 120)
            self.spawn_interval = max(280, SPAWN_INTERVAL - (self.level - 1) * 60 + jitter)

        # update all sprites and remove offscreen
        for s in list(self.all_sprites):
            if isinstance(s, Player):
                s.update(dt, now)
            else:
                # apply slowmo if active
                if now < self.slowmo_until:
                    s.update(dt * 0.45)
                else:
                    s.update(dt)

            # if below screen, remove
            if s.rect.top > HEIGHT + 140:
                if s in self.enemies:
                    # avoided enemy => score + coins
                    gained = int(10 + self.level * 2)
                    self.score += int(gained * self.skill_multiplier)
                    # small coin chance for avoiding
                    if random.random() < 0.05:
                        self.coins += 1
                s.kill()

        # collisions: enemies with player
        hits = pygame.sprite.spritecollide(self.player, self.enemies, False, pygame.sprite.collide_rect)
        for enemy in hits:
            # if shielded or nitro active and enemy not boss/big -> destroy enemy
            nitro_active = self.now < self.player.nitro_end_time
            if self.player.is_shielded or nitro_active:
                # damage enemy
                enemy.hp -= 1
                self.spawn_particles(enemy.rect.center, count=12, color=(255,120,60))
                try:
                    if 'hit' in self.sounds: self.sounds['hit'].play()
                except: pass
                if enemy.hp <= 0:
                    # reward
                    self.score += 25 * self.level
                    self.coins += 3
                    enemy.kill()
            else:
                # take damage
                self.player.lives -= 1
                self.player.hit_flash_until = now + 600
                self.spawn_particles(self.player.rect.center, count=10, color=(255,40,40))
                if 'hit' in self.sounds:
                    try: self.sounds['hit'].play()
                    except: pass
                # knockback
                self.player.rect.y -= 12
                if self.player.lives <= 0:
                    self.game_over()

        # collisions: player with powerups
        p_hits = pygame.sprite.spritecollide(self.player, self.powerups, True, pygame.sprite.collide_rect)
        for pu in p_hits:
            if pu.ptype == "shield":
                self.activate_shield(SHIELD_DURATION)
                self.score += 15
            elif pu.ptype == "slow":
                self.slowmo_until = now + SLOWMO_DURATION
            elif pu.ptype == "double":
                self.double_until = now + DOUBLE_SCORE_DURATION
            elif pu.ptype == "life":
                self.player.lives = min(5, self.player.lives + 1)
            elif pu.ptype == "coin":
                self.coins += 3
            # small effect sound
            try:
                if 'powerup' in self.sounds: self.sounds['powerup'].play()
            except: pass

        # level up
        target = 200 + self.level * 150
        if self.score + self.player.score >= target:
            self.level_up()

        # update speed based on level and slowmo/double effects
        # if slowmo active, enemies slower
        slow_mult = 0.6 if now < self.slowmo_until else 1.0
        self.enemy_speed = (BASE_ENEMY_SPEED + (self.level - 1) * ENEMY_ACCEL) * slow_mult

        # update score holder
        self.player.score = self.score

        # particles update
        for p in list(self.particles):
            p.update(dt)
            if not p.alive():
                self.particles.remove(p)

        # weather particles
        for wp in list(self.weather_particles):
            wp.update(dt)
            if not wp.alive():
                self.weather_particles.remove(wp)

        # day-night progression
        self.time_of_day += self.day_night_speed * (dt / 16.67)
        if self.time_of_day > 24: self.time_of_day -= 24

        # occasionally change weather
        if self.weather is None and random.random() < 0.0008:
            self.weather = random.choice([None, 'rain', 'snow', None, None])
            if self.weather:
                # spawn initial weather particles
                for _ in range(40):
                    self.spawn_weather_particle()

        # spawn weather particles gradually if active
        if self.weather in ('rain','snow') and random.random() < 0.6:
            self.spawn_weather_particle()

    def level_up(self):
        self.level += 1
        self.spawn_deque.append(self.level)
        # reward: every 3 level add life (but code originally set +0)
        if self.level % 3 == 0:
            self.player.lives = min(5, self.player.lives + 0)
        # speed bump
        self.enemy_speed += ENEMY_ACCEL * 3
        self.spawn_interval = max(280, self.spawn_interval - 80)
        # give coin reward
        self.coins += 5

    def activate_shield(self, duration_ms):
        self.player.is_shielded = True
        self.player.shield_end_time = self.now + duration_ms

    def spawn_particles(self, pos, count=8, color=(255,200,0)):
        for _ in range(count):
            vel = (random.uniform(-2,2), random.uniform(-1,4))
            p = Particle(pos, vel, lifetime=600+random.randint(0,400), color=color, size=random.randint(2,5))
            self.particles.append(p)

    def spawn_weather_particle(self):
        if self.weather == 'rain':
            x = random.randint(0, WIDTH)
            y = random.randint(-40, 0)
            vel = (0, random.uniform(8, 12))
            p = Particle((x,y), vel, lifetime=1600, color=(180,180,255), size=2)
            self.weather_particles.append(p)
        elif self.weather == 'snow':
            x = random.randint(0, WIDTH)
            y = random.randint(-40, 0)
            vel = (random.uniform(-1,1), random.uniform(1,3))
            p = Particle((x,y), vel, lifetime=2200, color=(240,240,255), size=3)
            self.weather_particles.append(p)

    def game_over(self):
        total_score = self.score + self.player.score
        if total_score > self.highscore:
            self.highscore = total_score
            save_text_file(HIGHSCORE_FILE, self.highscore)
        # update save coins/highscore
        save = load_json(SAVE_FILE, DEFAULT_SAVE)
        save['coins'] = self.coins
        save['highscore'] = self.highscore
        save_json(SAVE_FILE, save)
        # update leaderboard
        self.update_leaderboard(total_score)
        self.playing = False

    def update_leaderboard(self, score):
        lb = load_json(LEADERBOARD_FILE, [])
        name = "Player"
        lb.append({"name": name, "score": score})
        lb = sorted(lb, key=lambda x: x['score'], reverse=True)[:7]
        save_json(LEADERBOARD_FILE, lb)
        self.leaderboard = lb

    # ------------------ Draw methods -------------------
    def draw_road(self):
        # day-night interpolation
        tod = self.time_of_day
        is_night = tod < 6 or tod > 18
        bg = BG_COLOR_NIGHT if is_night else BG_COLOR_DAY
        road = ROAD_COLOR_NIGHT if is_night else ROAD_COLOR_DAY

        self.screen.fill(bg)
        road_rect = pygame.Rect(0, 40, WIDTH, HEIGHT - 80)
        pygame.draw.rect(self.screen, road, road_rect)

        # lane divider dashed
        dash_h = 30
        gap_h = 18
        offset = (self.road_offset // 4) % (dash_h + gap_h)
        for i in range(1, LANES):
            x = i * LANE_WIDTH
            y = 40 - offset
            while y < HEIGHT - 40:
                pygame.draw.rect(self.screen, LINE_COLOR, (x - 3, y, 6, dash_h))
                y += dash_h + gap_h

        # update road scrolling
        self.road_offset += self.road_speed
        if self.road_offset > 10000:
            self.road_offset = 0

    def draw_hud(self):
        # Score / Level / Lives / Coins / Highscore
        score_text = self.font.render(f"Skor: {self.score}", True, (255,255,255))
        lvl_text = self.font.render(f"Level: {self.level}", True, (255,255,255))
        lives_text = self.font.render(f"Nyawa: {self.player.lives}", True, (255,255,255))
        coins_text = self.font.render(f"Coins: {self.coins}", True, (255,255,255))
        hs_text = self.font.render(f"Highscore: {self.highscore}", True, (255,255,255))

        self.screen.blit(score_text, (12,8))
        self.screen.blit(lvl_text, (WIDTH//2 - 40, 8))
        self.screen.blit(lives_text, (WIDTH - 110, 8))
        self.screen.blit(coins_text, (WIDTH-120, HEIGHT-34))
        self.screen.blit(hs_text, (12, HEIGHT-34))

        # nitro meter
        nm_w = 120
        nm_h = 10
        x = WIDTH//2 - nm_w//2
        y = HEIGHT - 26
        pygame.draw.rect(self.screen, (40,40,40), (x, y, nm_w, nm_h))
        nm = int((self.player.nitro_meter / self.player.nitro_max) * nm_w)
        pygame.draw.rect(self.screen, (255,120,20), (x, y, nm, nm_h))
        # nitro cooldown indicator
        if self.now < self.player.nitro_ready_time:
            rem = (self.player.nitro_ready_time - self.now)/1000.0
            t = self.smallfont.render(f"Nitro CD: {rem:.1f}s", True, (255,255,255))
            self.screen.blit(t, (x, y-18))

        # shield indicator
        if self.player.is_shielded:
            rem = max(0, self.player.shield_end_time - self.now)/1000.0
            shield_text = self.smallfont.render(f"Shield: {rem:.1f}s", True, SHIELD_COLOR)
            self.screen.blit(shield_text, (WIDTH-160, HEIGHT-26))

        # double score
        if self.now < self.double_until:
            rem = (self.double_until - self.now)/1000.0
            t = self.smallfont.render(f"Double x2: {rem:.1f}s", True, (255,215,60))
            self.screen.blit(t, (WIDTH//2 - 60, 40))

    def draw(self):
        if not self.playing:
            self.screen.fill(BG_COLOR_DAY)
            # show menu / game over / shop
            if self.in_shop:
                self.draw_shop()
            else:
                self.draw_main_menu()
        else:
            # in-game draw
            self.draw_road()

            # draw sprites
            for sprite in self.all_sprites:
                self.screen.blit(sprite.image, sprite.rect)

            # shield overlay for player
            if self.player.is_shielded:
                r = max(self.player.rect.width, self.player.rect.height) // 2 + 8
                s = pygame.Surface((r*2, r*2), pygame.SRCALPHA)
                pygame.draw.circle(s, (255, 215, 0, 90), (r, r), r)
                self.screen.blit(s, (self.player.rect.centerx - r, self.player.rect.centery - r))

            # draw particle effects
            for p in self.particles:
                p.draw(self.screen)
            for wp in self.weather_particles:
                wp.draw(self.screen)

            # HUD
            self.draw_hud()

            # paused overlay
            if self.paused:
                pause_text = self.bigfont.render("PAUSED", True, (255,255,255))
                rect = pause_text.get_rect(center=(WIDTH//2, HEIGHT//2))
                self.screen.blit(pause_text, rect)

            # draw virtual buttons (transparent)
            self.draw_virtual_buttons()

        pygame.display.flip()

    def draw_main_menu(self):
        title = self.bigfont.render("MOBIL-MOBILAN (FULL)", True, (255,255,255))
        instr = self.font.render("Tekan ENTER atau klik untuk mulai", True, (255,255,255))
        sub = self.smallfont.render("Shift = Nitro | Space = Shield (demo) | P = Pause", True, (255,255,255))
        shop_hint = self.smallfont.render("Tekan S di menu untuk masuk Shop", True, (255,255,255))

        self.screen.blit(title, title.get_rect(center=(WIDTH//2, HEIGHT//2 - 80)))
        self.screen.blit(instr, instr.get_rect(center=(WIDTH//2, HEIGHT//2)))
        self.screen.blit(sub, sub.get_rect(center=(WIDTH//2, HEIGHT//2 + 32)))
        self.screen.blit(shop_hint, shop_hint.get_rect(center=(WIDTH//2, HEIGHT//2 + 64)))

        # show leaderboard
        y = HEIGHT//2 + 110
        lb = load_json(LEADERBOARD_FILE, [])
        if lb:
            self.screen.blit(self.smallfont.render("Leaderboard:", True, (255,255,255)), (WIDTH//2 - 60, y))
            for i,entry in enumerate(lb[:5]):
                t = self.smallfont.render(f"{i+1}. {entry['name']} - {entry['score']}", True, (255,255,255))
                self.screen.blit(t, (WIDTH//2 - 60, y + 20 + i*18))

        # show coins & highscore
        coins_txt = self.smallfont.render(f"Coins: {self.coins}", True, (255,255,255))
        hs_txt = self.smallfont.render(f"Highscore: {self.highscore}", True, (255,255,255))
        self.screen.blit(coins_txt, (12, HEIGHT-36))
        self.screen.blit(hs_txt, (12, HEIGHT-56))

    def draw_shop(self):
        self.screen.fill((28,28,38))
        title = self.bigfont.render("SHOP", True, (255,255,255))
        self.screen.blit(title, title.get_rect(center=(WIDTH//2, 60)))
        # show coin balance
        bal = self.font.render(f"Coins: {self.coins}", True, (255,255,255))
        self.screen.blit(bal, (20, 110))
        # shop items (skins)
        items = [
            {"id":"skin_red", "name":"Red Car", "price":30},
            {"id":"skin_gold", "name":"Gold Car", "price":60},
            {"id":"skin_blue", "name":"Sky Blue", "price":35},
        ]
        y = 150
        for it in items:
            rect = pygame.Rect(40, y, WIDTH-80, 56)
            pygame.draw.rect(self.screen, (40,40,60), rect, border_radius=8)
            self.screen.blit(self.font.render(it["name"], True, (255,255,255)), (60, y+8))
            ptxt = self.smallfont.render(f"Price: {it['price']}", True, (255,255,255))
            self.screen.blit(ptxt, (WIDTH-180, y+8))
            buy_rect = pygame.Rect(WIDTH-120, y+8, 72, 36)
            pygame.draw.rect(self.screen, (80,140,80), buy_rect, border_radius=6)
            self.screen.blit(self.smallfont.render("BUY", True, (255,255,255)), (WIDTH-100, y+16))
            it['rect'] = rect
            it['buy_rect'] = buy_rect
            y += 78

        # back hint
        back = self.smallfont.render("Klik ESC untuk kembali", True, (255,255,255))
        self.screen.blit(back, (20, HEIGHT-40))

    def draw_virtual_buttons(self):
        # Transparent rectangles for touchscreen control guidance
        alpha = 90
        s = pygame.Surface((self.virtual["left"].w, self.virtual["left"].h), pygame.SRCALPHA)
        s.fill((60,60,60,alpha))
        self.screen.blit(s, (self.virtual["left"].x, self.virtual["left"].y))
        s2 = pygame.Surface((self.virtual["right"].w, self.virtual["right"].h), pygame.SRCALPHA)
        s2.fill((60,60,60,alpha))
        self.screen.blit(s2, (self.virtual["right"].x, self.virtual["right"].y))
        s3 = pygame.Surface((self.virtual["nitro"].w, self.virtual["nitro"].h), pygame.SRCALPHA)
        s3.fill((140,60,20,alpha))
        self.screen.blit(s3, (self.virtual["nitro"].x, self.virtual["nitro"].y))
        # labels
        l = self.smallfont.render("LEFT", True, (255,255,255))
        r = self.smallfont.render("RIGHT", True, (255,255,255))
        n = self.smallfont.render("NITRO", True, (255,255,255))
        self.screen.blit(l, (self.virtual["left"].centerx-20, self.virtual["left"].centery-8))
        self.screen.blit(r, (self.virtual["right"].centerx-24, self.virtual["right"].centery-8))
        self.screen.blit(n, (self.virtual["nitro"].centerx-28, self.virtual["nitro"].centery-8))

    # ------------------ Quit -------------------
    def quit(self):
        # save progress
        save = load_json(SAVE_FILE, DEFAULT_SAVE)
        save['coins'] = self.coins
        save['selected_skin'] = self.player.skin
        save['highscore'] = self.highscore
        save_json(SAVE_FILE, save)

        self.running = False
        try:
            pygame.quit()
        except:
            pass
        raise SystemExit

# -------------- Entry -----------------------
if __name__ == "__main__":
    game = Game()
    try:
        game.run()
    except SystemExit:
        pass


pygame 2.6.1 (SDL 2.28.4, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
